In [ ]:
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
from google.colab import drive
# Cambiar por el folder donde se encuentren los datos.
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/chest_xray'
# Elegimos el tamaño del Batch, el tamaño de imagen y los pasos por epoca 
batch_size = 16
image_shape = (128, 128)
STEPS_PER_EPOCH = 20


Mounted at /content/drive


# Cargar la Red

En este caso para las capas convolucionales usamos la RedNet50 para obtener ciertos rasgos de la imagen

In [ ]:
from keras.applications import ResNet50, EfficientNetB3

retnet50 = ResNet50(
  weights='imagenet',
  #Fijamos el include top en falso para no cargar capas que no usaremos. 
  include_top=False,
  input_shape=(*image_shape, 3)
)

94765736/94765736 [==============================] - 4s 0us/step


# Generador de Imagenes

In [ ]:

datagen = ImageDataGenerator(
  rescale=1/255,
  preprocessing_function=preprocess_input)

#Generamos los directorios con los subdirectorios que definen las clases 
train_generator = datagen.flow_from_directory(
    folder + '/train',
    target_size = image_shape,
    batch_size  = batch_size,
    shuffle = True,
    class_mode  = 'binary'
)

test_generator = datagen.flow_from_directory(
    folder + '/test',
    target_size = image_shape,
    batch_size  = batch_size,
    class_mode  = 'binary'
)

eval_generator = datagen.flow_from_directory(
    folder + '/val',
    target_size = image_shape,
    batch_size  = batch_size,
    class_mode  = 'binary'
)
#Notemos que la base datos no está balanceada 

Found 5217 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


# Generar red neuronal

Creamos las siguientes capas densas de la red congelamos las capas convolucionales de la rednet50 (Backbone
)

In [ ]:
from keras import models
from keras import layers
import tensorflow as tf
#Creamos las capas densas 
model = models.Sequential()
model.add(retnet50) 
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
#Congelamos las capas convolucionales 
retnet50.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4194432   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 27,792,513
Trainable params: 4,204,801
Non

In [ ]:
#Compilamos el modelo con la función de perdida de entropía cruzada  y el optimizador adam
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['acc']
)
#Guardar el mejor modelo 
model_route = '/content/drive/MyDrive/best_model.h5'
save_model = tf.keras.callbacks.ModelCheckpoint(
  filepath=model_route,
  save_weights_only=True,
  verbose=0, 
  save_best_only=True, 
  monitor='val_loss', 
  mode='min'
)

In [ ]:
#Dado que la base de datos no esta balanceada, calculamos unos pesos especiales 
pesos = {
  0 : train_generator.classes.shape[0] / np.count_nonzero(train_generator.classes==0), 
  1 : train_generator.classes.shape[0] / np.count_nonzero(train_generator.classes)
}

print(pesos)

{0: 3.8874813710879286, 1: 1.3463225806451613}


In [ ]:
model.fit(
  train_generator,
  steps_per_epoch = STEPS_PER_EPOCH, 
  epochs          = 25,
  validation_data = eval_generator,
  validation_steps= 1,
  class_weight = pesos,   
  verbose         = 2,
  callbacks = [save_model]
)

Epoch 1/25


# Descongelamos el Backbone


In [ ]:
#Descongelamos y entrenamos las ultimas 25 capas de la red convolucional 
retnet50.trainable = True
layer_names = [layer.name for layer in retnet50.layers]
for layer in retnet50.layers:
    if layer.name in layer_names[-25:]:
        layer.trainable = True
    else:
        layer.trainable = False
        
retnet50.summary()

In [ ]:
#Compilamos el modelo con la función de perdida de entropía cruzada  y el optimizador adam
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

model.fit(
  train_generator,
  steps_per_epoch = STEPS_PER_EPOCH, 
  epochs          = 25,
  validation_data = eval_generator,
  validation_steps= 1,
  class_weight = pesos,
  verbose         = 2,
  callbacks = [save_model]
)

# Full tuning 



In [ ]:
#Descongelamos todas las capas de la red neuronal.
for layer in retnet50.layers:
  layer.trainable = True
      
retnet50.summary()

In [ ]:
#Compilamos el modelo con la función de perdida de entropía crusada  y el optimizador adam
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

model.fit(
  train_generator,
  steps_per_epoch = STEPS_PER_EPOCH, 
  epochs          = 20,
  validation_data = eval_generator,
  validation_steps= 1,
  class_weight = pesos,
  verbose         = 2,
  callbacks = [save_model]
)

In [ ]:
model.save('ModeloTa4.h5')

# Resultados

In [ ]:
model.evaluate(test_generator)


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

y_true_labels = test_generator.classes
y_pred_labels = np.round(model.predict(test_generator))

print(f"""
Test results
Accuracy : {accuracy_score(y_true_labels, y_pred_labels)}
Recall : {recall_score(y_true_labels, y_pred_labels)}
Specificity : {recall_score(y_true_labels, y_pred_labels, pos_label = 0)}
Precision : {precision_score(y_true_labels, y_pred_labels)}
""")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

C = confusion_matrix(y_true_labels, y_pred_labels)

f, ax = plt.subplots(figsize=(11, 9))
sns.set()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
ax = sns.heatmap(C, cmap=cmap, square=True, linewidths=.5)
ax.set_title('Matriz de Confusión')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_true_labels, y_pred_labels)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

Se intentaron algunas variantes para mejorar el modelo pero todo termina de forma similar. Me parece que la red esta prediciendo muchas veces una clase. Se intento guardar el mejor modelo para el validation encontrado, dejar que corra, aumentar el numero de neuronas en la densa, diferentes optimizadores y todas los intentos tuvieron resultados similares que no fueron muy buenos. Me parece  que el desbalance de los datos afecta bastante a la red neuronal aun cuando se ajustaron los pesos.